<h1>Python + Selenium </h1>

* Instalar o Selenium
#abrir o prompt e dar pip install selenium
* Baixar o WebDriver (chromedriver)
<br>#webdriver é um arquivo que vai permitir o python a integrar ao navegador; para isso é necessário pegar o webdriver dos navegadores distintos que vc irá usar.
<br>#colocar o arquivo onde está instalado o python(python.exe), no caso em C:\Users\User\anaconda3\

In [ ]:
from selenium import webdriver
navegador = webdriver.Chrome()
##navegador.get vai até uma pagina
site_investimento = ("https://fiis.com.br/")
#print(site_investimento)
navegador.get(site_investimento)
##procura pelo xpath e clica
#navegador.find_element("xpath", '//*[@id="fii-section"]/div/div/div[2]/div[3]/div/div[1]/div[2]/div/span').click()

Depois de criarmos a estrutura do webdriver para acessar a página, iremos importar o Pandas e criaremos um dataFrame com os ativos que precisaremos pesquisar. Esses ativos servirão também de sufixos para a url do site_investimento.

Poderíamos fazer isso importando um csv com a lista.

In [ ]:
import pandas as pd
#Criaremos um dataframe coms os ativos que vão ser pesquisados
ativos = pd.DataFrame({"nome":["AFHI11","SDIL11","XPML11","IRDM11","RBVA11","RBED11","FLMA11","TRXF11","HGRU11","TRBL11","KNSC11","MALL11"]})
ativos

,nome
0,AFHI11
1,SDIL11
2,XPML11
3,IRDM11
4,RBVA11
5,RBED11
6,FLMA11
7,TRXF11
8,HGRU11
9,TRBL11


Depois de ter criado a estrutura de ativos que serão pesquisados, vamos fazer uma iteração que vai acessar o site_investimento/ com o sufixo dos Ativos criados no bloco anterior.

Para isso importaremos da biblioteca do webdriver o **By**, para podermos achar o elemento do xPath da página. Como o que vamos puxar agora é uma tabela, também poderia ser feito através do CLASSNAME.

Ao puxar o elemento, vamos transformá-lo em um texto e começaremos a tratá-lo, transformando no Dataframe **df**.


In [ ]:
from selenium.webdriver.common.by import By
#Importaremos a biblioteca By
#from io import StringIO     # Usamos StringIO para simular um arquivo em memória

integra_df = pd.DataFrame()
#Faremos uma iteração aqui, criando o indicador IND, procurando pelo index de ativos(ativos.index)
for ind in ativos.index:
    pesquisa = ativos.loc[ativos.index[ind]]    #pesquisa vai guardar as infos de cada index, começando do 0
    #print(pesquisa)                             #Veremos algo do tipo: nome    AFHI11    Name: 0, dtype: object 
    pesquisa = pesquisa['nome']                 #Como só interessa a coluna nome, iremos filtrar por ela
    #print(pesquisa)                             #aqui teremos só a coluna nome do index, que será AFHI11
    url = (site_investimento+pesquisa)           #Puxando o Site_investimento (fiis.com),iremos criar a string com o nome
    #print(url)
    navegador.get(url)                           #Puxaremos então a url  fiis.com.br/nome atual
    #print('Novo CICLO!  ',ind)
    
    #Busca Elemento Na Tabela
    tabela = navegador.find_element(By.XPATH, '//*[@id="carbon_fields_fiis_dividends-2"]/div[2]/div[2]')
    #tentar procurar com a Classe 
    #tabela = navegador.find_element(By.CLASSNAME, 'dividends-table-container')
    texto = tabela.text
    texto = texto.strip().split('\n')
    
    df = pd.DataFrame([x.split(';') for x in texto if x])
    ###
    
    ##Referencia Nome da Tabela, Com o Nome do FII
    nome_da_tabela = df[0][0]
    
    ##Busca Ultimo Rendimento Apenas
    ult_rend = navegador.find_element(By.XPATH, '//*[@id="carbon_fields_fiis_header-2"]/div/div/div[1]/div[2]/div/div[2]/p[1]/b')
    #print(pesquisa + ' Rendeu no Ultimo mês ' + ult_rend.text)
    
    ######### FAZ A ITERAÇÂO ############
    de = df #Dropa a primeira linha
    i = 0
    c = 0
    coluna_df = pd.DataFrame()
    result_df = coluna_df
    
    for ind in de.values:
        if i==0:
            #print('Dropar',ind)
            pass
        elif i > 0 and i < 6 :
            #print('Cabeçalho', ind)
            coluna_df[ind] = 'None'
            #print(coluna_df)
        else:
            if i ==6:
                result_df = coluna_df
            #print(ind, i)
            #print(c)
            ## Atribua os valores pelo índice da coluna
            coluna_df[coluna_df.columns[c]] = ind
            #print('Coluna_df : '+coluna_df)
            #print('Coluna_df Posição : '+coluna_df.columns[c])
            #print('Coluna_df Valor : '+ind)
            c = c + 1
            if c >= 5:
                c = 0
                result_df = result_df.append(coluna_df, ignore_index=True)
            #print('Result_df = '+result_df)
        i=i+1       
    #Tiramos aqui a primeira linha da tabela, que contem lixo de informação
    result_df = result_df.drop(range(1))
    #Agora, criamos uma coluna inteira com o Nome do Fii correspondente
    result_df = result_df.assign(NomeFii=pesquisa)
    #Por fim, associamos uma nova Dataframe que recebe a Dataframe Result, podendo assim armazenar todos os dados.
    integra_df = integra_df.append(result_df, ignore_index=True)

AFHI11 Rendeu no Ultimo mês 1,05


C:\Users\User\AppData\Local\Temp\ipykernel_7688\4254564762.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(coluna_df, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_7688\4254564762.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  integra_df = integra_df.append(result_df, ignore_index=True)


SDIL11 Rendeu no Ultimo mês 0,75


C:\Users\User\AppData\Local\Temp\ipykernel_7688\4254564762.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(coluna_df, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_7688\4254564762.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  integra_df = integra_df.append(result_df, ignore_index=True)


XPML11 Rendeu no Ultimo mês 0,85


...

C:\Users\User\AppData\Local\Temp\ipykernel_7688\4254564762.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(coluna_df, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_7688\4254564762.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  integra_df = integra_df.append(result_df, ignore_index=True)


KNSC11 Rendeu no Ultimo mês 0,75


C:\Users\User\AppData\Local\Temp\ipykernel_7688\4254564762.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(coluna_df, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_7688\4254564762.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  integra_df = integra_df.append(result_df, ignore_index=True)


MALL11 Rendeu no Ultimo mês 0,85


C:\Users\User\AppData\Local\Temp\ipykernel_7688\4254564762.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(coluna_df, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_7688\4254564762.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  integra_df = integra_df.append(result_df, ignore_index=True)


In [ ]:
##Joga o Dataframe inteiro em um arquivo consolidado csv 
integra_df.to_csv(r'C:\Users\User\Downloads\PlanilhaConsolidado.csv', sep=';', encoding='ANSI',index=False)
integra_df

# Colunas ['Data Base', 'Data Pagamento', 'Cotação Base', 'Dividend Yield', 'Rendimento', 'NomeFii'])


,Data Base,Data Pagamento,Cotação Base,Dividend Yield,Rendimento,NomeFii
0,15.09.2023,22.09.2023,"R$ 98,23","1,07%","R$ 1,05",AFHI11
1,14.08.2023,21.08.2023,"R$ 97,87","1,07%","R$ 1,05",AFHI11
2,14.07.2023,21.07.2023,"R$ 98,75","1,11%","R$ 1,10",AFHI11
3,15.06.2023,22.06.2023,"R$ 97,97","1,12%","R$ 1,10",AFHI11
4,15.05.2023,22.05.2023,"R$ 95,35","1,15%","R$ 1,10",AFHI11
...,...,...,...,...,...,...
715,30.05.2018,14.06.2018,"R$ 99,42","0,62%","R$ 0,62",MALL11
716,30.04.2018,15.05.2018,"R$ 105,30","0,58%","R$ 0,61",MALL11
717,29.03.2018,13.04.2018,"R$ 102,95","0,65%","R$ 0,67",MALL11
718,28.02.2018,14.03.2018,"R$ 103,20","0,48%","R$ 0,50",MALL11


In [ ]:
#Leremos o CSV (por desencargo de exercicio, poderia ser feito ja com o integra_df 

ler = pd.read_csv(r'C:\Users\User\Documents\PlanilhaConsolidado.csv', encoding='ANSI',sep=';') 
#ler
for ind in ativos.index:
    pesquisa = ativos.loc[ativos.index[ind]]    #pesquisa vai guardar as infos de cada index, começando do 0
    #print(pesquisa)                             #Veremos algo do tipo: nome    AFHI11    Name: 0, dtype: object 
    pesquisa = pesquisa['nome']                 #Como só interessa a coluna nome, iremos filtrar por ela
    FIIespecifico = ler.loc[ler['NomeFii'] == pesquisa]
    FIIespecifico



,Data Base,Data Pagamento,Cotação Base,Dividend Yield,Rendimento,NomeFii
651,29.09.2023,16.10.2023,"R$ 117,69","0,72%","R$ 0,85",MALL11
652,31.08.2023,15.09.2023,"R$ 117,31","0,72%","R$ 0,85",MALL11
653,31.07.2023,14.08.2023,"R$ 115,50","0,69%","R$ 0,80",MALL11
654,30.06.2023,14.07.2023,"R$ 117,48","0,68%","R$ 0,80",MALL11
655,31.05.2023,15.06.2023,"R$ 109,50","0,71%","R$ 0,78",MALL11
...,...,...,...,...,...,...
715,30.05.2018,14.06.2018,"R$ 99,42","0,62%","R$ 0,62",MALL11
716,30.04.2018,15.05.2018,"R$ 105,30","0,58%","R$ 0,61",MALL11
717,29.03.2018,13.04.2018,"R$ 102,95","0,65%","R$ 0,67",MALL11
718,28.02.2018,14.03.2018,"R$ 103,20","0,48%","R$ 0,50",MALL11
